In [ ]:
import datajoint as dj
import numpy as np

In [ ]:
import calcification_Module as cm

In [ ]:
import os
import pathlib

##write the OFF file for the neuron
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [ ]:
#connect to the database and pull down Cleansed Mesh ones and generate the resurfacing of these
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [ ]:
ta3p100.CleansedMesh()

In [ ]:
segment_id = 648518346342806978
label_key = dict(segment_id=segment_id)
vertices,triangles = (ta3p100.CleansedMesh & label_key).fetch1("vertices","triangles")


In [ ]:
#generate the off file for this
file_name_path,file_name,file_loc = write_Whole_Neuron_Off_file(segment_id,vertices.tolist(),triangles.tolist())

In [ ]:
#generate the Poisson surface reconstruction (need to run script )
#### THIS WILL HAVE TO BE A MANUAL STEP UNTIL I FIX IT
import os
import subprocess
os.system("./setup_meshlabserver.sh")

#execfile


In [ ]:
def make_executable(path):
    mode = os.stat(path).st_mode
    mode |= (mode & 0o444) >> 2    # copy R bits to X
    os.chmod(path, mode)

In [ ]:
#try running a meshlabserver command write here
new_string = ("meshlabserver -i ./temp/neuron_" + str(segment_id) +
           ".off -o ./temp/neuron_" + str(segment_id) + "_new.off " + 
           "-s remeshing_script_5.mlx")
#os.system(new_string)
#print(new_string)






In [ ]:
commands_to_run = ['function meshlabserver() { xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@; }',
                  'export -f meshlabserver']

path = 'neuron-' + str(segment_id) + '.sh'
with open(path, 'w+') as f:           # umask in effect when file is created
    f.write('#!/bin/bash\n')
    f.write('echo "hello world"\n')
    f.write(commands_to_run[0]+"\n")
    f.write(commands_to_run[1]+"\n")
    f.write(new_string)

make_executable(path)
os.system("./"+path)

In [ ]:
#run the skeletonization module:
import calcification_Module as cm
dir(cm)

import time

import os
from pathlib import Path
filename = "Voxelized_Mesh_Resurfaced"
total_path = os.getcwd() + "/" + filename


In [ ]:
start_time = time.time()
return_value = cm.calcification(total_path)
print(return_value)
end_time = time.time()
print(f"End time = {time.time()-start_time}")